In [ ]:
import xfelmay2019 as xfel

# Plot options
import matplotlib.pyplot as plt

# Import required libraries
import numpy as np
import pyqtgraph as pg

# Import karabo libraries
import karabo_bridge as kb
import karabo_data as kd

# Global Variables to define

In [ ]:
comissioningPath='/gpfs/exfel/exp/SQS/201921/p002430/raw'
methanePath ='/gpfs/exfel/exp/SQS/201802/p002176/raw'



# ***WILL NEED TO BE UPDATED WITH NEON AND METHANE CLUSTERS***

tofrange = (260000,285000)
pixels = np.arange( tofrange[0], tofrange[1] )

#Regions of interest for Xenon cluster spectra.  
onePlus=(280000 - 1000,280000 + 1500)       
lightPeak=(266000 - 200,266000 + 100)       
highCharge=(269000 - 200, 276000)

# General 

In [ ]:
#  help(xfel) lists all functions in xfel

#this gives pixels (the x-axis) for the TOF spectra, TOF spectra array, and the train ID for a given run
pixels, tof400, tids400= xfel.getRunTOF( 400, comissioningPath, tofrange=(260000,285000) )

runNumber = 400

#this gives an array of train ideas for a given run
trainIds = xfel.getTrainIds(runNumber,comissioningPath)

# Plotting a Single TOF Spectrum with ROI defined

In [ ]:
#specify run data
run = xfel.runFormat( 400 )  

#accessing all data of a given run
runData = kd.RunDirectory(comissioningPath+run)

#getting the TOF data from the run
tof400 = np.asarray(runData.get_array( 'SQS_DIGITIZER_UTC1/ADC/1:network','digitizers.channel_1_A.raw.samples' ))[ : , tofrange[0]:tofrange[1] ]


### Example 1: plotting a single TOF spectrum - average spectrum of run 400

In [ ]:
#averaging all TOF data of a given run
avgtof = xfel.averageTOF(tof400) 

#plotting a single TOF spectrum
xfel.plotTOF(pixels, avgtof)

###  Example 2: plotting the average of all TOF spectra from run 400, with ROI

In [ ]:
#averaging all TOF data of a given run
avgtof = xfel.averageTOF(tof400) 

#plotting the averaged TOF data with ROI's labeled
xfel.showROIs( pixels, avgtof, 
         onePlus=onePlus, 
         lightPeak=lightPeak,
         highCharge=highCharge )


###  Example 3: plotting the average of the brightest TOF spectra from run 400, with ROI

In [ ]:
#averaging the brightest TOF data of a given run
brighttof = xfel.averageBrightestTOFs(pixels, tof400) 

#plotting the averaged bright TOF data with ROI's labeled
xfel.showROIs( pixels, brighttof, 
         onePlus=onePlus, 
         lightPeak=lightPeak,
         highCharge=highCharge )

# Comparing TOF spectra from multiple runs

    Looking at the average of the brightest spectra and labeling with respect to pressure

In [ ]:
#extracting pixel axis, TOF data, and TID 
pixels, tof400, tids400= xfel.getRunTOF( 400, comissioningPath, tofrange=(260000,285000) )
pixels, tof401, tids401= xfel.getRunTOF( 401, comissioningPath, tofrange=(260000,285000) )
pixels, tof404, tids404= xfel.getRunTOF( 404, comissioningPath, tofrange=(260000,285000) )

### Averaging the brightest TOF spectra from specified runs

In [ ]:
#getting average TOF spectra from the brightest of each of the specfied runs
bright400 = xfel.averageBrightestTOFs(pixels, tof400)
bright401 = xfel.averageBrightestTOFs(pixels, tof401)
bright404 = xfel.averageBrightestTOFs(pixels, tof404)

### Can compare in waterfall plots

In [ ]:
#plotting these averages as waterfall
xfel.waterfallTOFs( pixels, [bright404, bright400,bright401], labels=['6 bar, 404','10 bar, 400','13.9 bar, 401'] )

### Can compare overlaying spectra

In [ ]:
#plotting these averages, overlayed
xfel.overlayTOFs( pixels, [bright404,bright400,bright401], labels=['6 bar, 404','10 bar, 400','13.9 bar, 401'] )

# Make a waterfall plot of the brightest TOF spectra for run 437, labeled with TID

In [ ]:
pixels, tof437, trainIds =xfel.getRunTOF( 437, comissioningPath, tofrange=(260000,285000) )

In [ ]:
# Choose the region of interest to use to sort the brightest
# can set onePlus to onePlus, lightPeak, or highCharge, which are defined at the top
# in this case, we are sorting by the X+ peak

#help(xfel.waterfallBrightest_labelByTrainId)

interestingTIDS437 = xfel.waterfallBrightest_labelByTrainId( pixels, tof437, np.array(trainIds), nbright=100, integrateAt=onePlus)

# Make a waterfall plot of the brightest TOF spectra for run 437, labeled with TID

In [ ]:
pixels, tof437, trainIds =xfel.getRunTOF( 437, comissioningPath, tofrange=(260000,285000) )

In [ ]:
interesting437 = xfel.waterfallBrightest(pixels, tof437, integrateAt=onePlus)

# Waterfall of Brightest TOFs averaged, labeled by run number

In [ ]:
runRange = np.arange(413, 415)

#getting averages of brightest TOFs from the runs specified above
avgtogs = xfel.getBrightAvgRunsTOF (runRange, comissioningPath, tofrange, integrateAt = highCharge)

In [ ]:
#making a waterfall plot of the average TOFS, labeled by run number

#remove waterfallDelta =, and traces will separate automatically such that they are not overlapping
xfel.waterfallTOFs(pixels, avgtogs, labels = [str(arun) for arun in runRange], waterfallDelta = -1000)

In [ ]:
help(xfel.waterfallTOFs)

# Plot against scan variable

In [ ]:
#specifying range of runs
runRange =np.array([263, 266, 267, 268])

#choosing scan variable to plot against
scanVar = np.array([np.mean(xfel.getChamberHeight( run , comissioningPath )[1]) for run in runRange])


In [ ]:
#index of sorted list
inds =  np.argsort(scanVar)

In [ ]:
#averaging the brightest TOFs over a range of runs
avgtofs = xfel.getBrightAvgRunsTOF( runRange, comissioningPath, tofrange, integrateAt=highCharge )

In [ ]:
#sorting the averaged TOFs by specified variable in a waterfall plot, and labeling run number
xfel.waterfallTOFs( pixels, np.array(avgtofs)[inds,:], labels=['%d, %0.3f'%(arun, avar) for arun,avar in zip(runRange, scanVar[inds])], waterfallDelta = -100 )

In [ ]:
#plotting integrated region of interest (highCharge, light peak, and onePlus) against the scan variable
plt.plot( scanVar, tofROI( pixels, np.array(avgtofs), roi=highCharge ))
plt.ylabel('Integrated high charge')
plt.xlabel('Scan height')

# Plot against scan variable within a single run

In [ ]:
#specifying range of runs
runNum = 107

#choosing scan variable to plot against
stid, scanVar = xfel.getPulseDelay( runNum , methanePath )

In [ ]:
ptid, PEs = xfel.getPulseEnergies( runNum , methanePath )

In [ ]:
plt.hist(PEs,bins=100);

In [ ]:
sub_ptid = ptid[(PEs>2400)&(PEs<2600)]
sub_PEs  = PEs[(PEs>2400)&(PEs<2600)]

In [ ]:
_,sinds,pinds = np.intersect1d( stid, sub_ptid,  return_indices=True)

In [ ]:
scanVar = scanVar[sinds]
stid = stid[sinds]

In [ ]:
plt.hist(scanVar, bins=100);
plt.xlabel('Scan variable')
plt.ylabel('Counts')

# Identify unique scan positions

In [ ]:
uniqueVar = np.unique(scanVar)

# Get TOF positions

In [ ]:
pixels, tof, ttid = xfel.getRunTOF( runNum, methanePath, tofrange=(260000,285000) )

# Intersect the tids for the scan var and the tof

In [ ]:
_,sinds,tinds = np.intersect1d( stid, ttid,  return_indices=True)
subtof = tof[tinds,:]
subvar = scanVar[sinds]

In [ ]:
avgtofs = []
for el in uniqueVar:
    inds = (subvar == el)
    avgtofs.append( xfel.averageBrightestTOFs(pixels, subtof[inds,:], integrateAt=highCharge, behlkeAt=265000))

# Waterfall the average brightest TOFs versus waterfall

In [ ]:
xfel.waterfallTOFs( pixels, np.array(avgtofs), labels=['%0.1f'%(avar) for avar in uniqueVar], waterfallDelta = -50 )
plt.ylim([0,-400])
plt.title('Average brightest TOFs vs. pulse delay')


In [ ]:
avgtofs = []
for el in uniqueVar:
    inds = (subvar == el)
#     avgtofs.append( xfel.averageBrightestTOFs(pixels, subtof[inds,:], integrateAt=highCharge, behlkeAt=265000))
    avgtofs.append( xfel.averageTOF(subtof[inds,:]))

# Waterfall the average TOFs versus waterfall

In [ ]:
xfel.waterfallTOFs( pixels, np.array(avgtofs), labels=['%0.1f'%(avar) for avar in uniqueVar], waterfallDelta = -50 )
plt.ylim([0,-400])
plt.title('Average TOFs vs. pulse delay')


# Plot integrated high charge vs. scan delay

In [ ]:
plt.plot( uniqueVar, xfel.tofROI( pixels, np.array(avgtofs), roi=highCharge ))
plt.ylabel('Integrated high charge')
plt.xlabel('Scan delay')

# Plot Pulse energy versus scan delay

In [ ]:
plt.scatter(scanVar, sub_PEs[pinds] )
plt.ylabel('Pulse energy')
plt.xlabel('Scan delay')

# XGM data for run

In [ ]:
Path ='/gpfs/exfel/exp/SQS/201802/p002176/raw'

run = xfel.runFormat( 99 )
runData = kd.RunDirectory(Path+run)
pulse_E = np.asarray(runData.get_array('SA3_XTD10_XGM/XGM/DOOCS:output','data.intensityTD' ))

train=pulse_E[1,:]
pos=max(enumerate(train), key=(lambda a: a[1]))

pulse_E=pulse_E[:,pos[0]]
plt.plot(pulse_E)
plt.show()
#np.average(pulse_E)